### Gemini APIの前準備

In [ ]:
# パッケージのインストール
!pip install -q -U google-generativeai

In [1]:
# from google.colab import userdata
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
# 環境変数の準備 (左端の鍵アイコンでGOOGLE_API_KEYを設定)
# GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
# genai.configure(api_key=GOOGLE_API_KEY)

### Automatic Function Calling

In [1]:
def add(a:float, b:float):
    """returns a + b."""
    return a+b

def subtract(a:float, b:float):
    """returns a - b."""
    return a-b

def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

def divide(a:float, b:float):
    """returns a / b."""
    return a*b

In [4]:
# モデル生成時に関数リストを指定
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash",
    tools=[add, subtract, multiply, divide]
)

In [5]:
# チャットの作成
chat = model.start_chat(
    enable_automatic_function_calling=True
)

In [11]:
# 関数の利用を促す質問応答
response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?"
)
response.text

'合計で2508個のミトンがあります。'

In [13]:
# 会話履歴の確認
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("--")

user -> [{'text': '私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?'}]
--
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--
model -> [{'text': '合計で2508個のミトンがあります。'}]
--


In [28]:
for content in chat.history:
    print(type(content.parts[0]).to_dict(content.parts[0]))
    break


{'text': '私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?'}


### Tool Config

In [29]:
from google.generativeai.types import content_types
from collections.abc import Iterable

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Function Calling ModeでTool Configを作成"""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

In [30]:
# 会話履歴のクリア
chat.history.clear()

# 推論の実行
response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?",
    tool_config=tool_config_from_mode("none") # NONE
)

# 会話履歴の確認
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("--")

user -> [{'text': '私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?'}]
--
model -> [{'text': '猫は 57 匹いて、それぞれにミトンが 44 個あるので、ミトンの総数は 57 x 44 = 2508 個になります。\n'}]
--


In [31]:
# 会話履歴のクリア
chat.history.clear()

# 推論の実行
response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?",
    tool_config=tool_config_from_mode("auto") # AUTO
)

# 会話履歴の確認
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("--")

user -> [{'text': '私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?'}]
--
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--
model -> [{'text': '57匹の猫がそれぞれ44個のミトンを持っている場合、合計で2508個のミトンがあります。 \n'}]
--


In [ ]:
# 会話履歴のクリア
chat.history.clear()

# 推論の実行
response = chat.send_message(
    "私は57匹の猫を飼っていて、それぞれが44個のミトンを持っています。ミトンは合計で何個になりますか?",
    tool_config=tool_config_from_mode("any", ["multiply"]) # ANY
)

# 会話履歴の確認
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("--")

### Manual Function Calling

In [32]:
# 気温の取得 (ダミー)
def get_temperature(location: str):
    """get current temperature."""
    if location == "東京":
        return "20度"
    else:
        return "10度"

In [33]:
# 関数を辞書で管理
functions = {
    "get_temperature": get_temperature,
}

In [34]:
# モデルの準備
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-flash",
    tools=functions.values(), # dict.values() => iterだから？
)

In [43]:
a = {"a": 2}
for a in a.values():
    print(a)

2


In [57]:
# 関数の利用を促す質問応答
prompt = "現在の東京の気温は？"
response = model.generate_content(prompt)
response.candidates[0].content.parts

[function_call {
  name: "get_temperature"
  args {
    fields {
      key: "location"
      value {
        string_value: "東京"
      }
    }
  }
}
]

In [47]:
# 多分argsでアンラップしている？
test = {'location': '東京'}
get_temperature(**test)

'20度'

In [50]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_temperature",
                  "args": {
                    "location": "\u6771\u4eac"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEG

In [62]:
response.candidates[0].content.parts[0].function_call

name: "get_temperature"
args {
  fields {
    key: "location"
    value {
      string_value: "東京"
    }
  }
}

In [53]:
import google.ai.generativelanguage as glm

# 関数呼び出し
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

# function_responsesの生成
function_responses = []
for part in response.parts:
    # 関数の使用を選択したかどうかの確認
    if part.function_call:
        # 関数呼び出しの実行
        result = call_function(part.function_call, functions)

        # function_responseの生成
        function_response = glm.Part(function_response=glm.FunctionResponse(
            name=part.function_call.name,
            response={"result": result}
        ))
        function_responses.append(function_response)
print(function_responses)

[function_response {
  name: "get_temperature"
  response {
    fields {
      key: "result"
      value {
        string_value: "20度"
      }
    }
  }
}
]


In [63]:
# 会話履歴の作成
messages = [
    {'role':'user',
     'parts': [prompt]},
    {'role':'model',
     'parts': response.candidates[0].content.parts},
    {'role':'user',
     'parts': function_responses}
]

# 質問応答
response = model.generate_content(messages)
print(response.text)

現在の東京の気温は20度です。 



# Parallel Function Calling

In [107]:
# 関数の利用を促す質問応答
prompt = "現在の東京と大阪の気温は？"
response = model.generate_content(prompt)
response.candidates[0].content.parts

[function_call {
  name: "get_temperature"
  args {
    fields {
      key: "location"
      value {
        string_value: "東京"
      }
    }
  }
}
, function_call {
  name: "get_temperature"
  args {
    fields {
      key: "location"
      value {
        string_value: "大阪"
      }
    }
  }
}
]

In [108]:
import google.ai.generativelanguage as glm

# 関数呼び出し
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

# function_responsesの生成
function_responses = []
for part in response.parts:
    # 関数の使用を選択したかどうかの確認
    if part.function_call:
        # 関数呼び出しの実行
        result = call_function(part.function_call, functions)

        # function_responseの生成
        function_response = glm.Part(function_response=glm.FunctionResponse(
            name=part.function_call.name,
            response={"result": result}
        ))
        function_responses.append(function_response)
print(function_responses)

[function_response {
  name: "get_temperature"
  response {
    fields {
      key: "result"
      value {
        string_value: "20度"
      }
    }
  }
}
, function_response {
  name: "get_temperature"
  response {
    fields {
      key: "result"
      value {
        string_value: "10度"
      }
    }
  }
}
]


In [109]:
a = function_responses
a[::-1]

[function_response {
   name: "get_temperature"
   response {
     fields {
       key: "result"
       value {
         string_value: "10度"
       }
     }
   }
 },
 function_response {
   name: "get_temperature"
   response {
     fields {
       key: "result"
       value {
         string_value: "20度"
       }
     }
   }
 }]

In [111]:
# 会話履歴の作成
messages = [
    {'role':'user',
     'parts': [prompt]},
    {'role':'model',
     'parts': response.candidates[0].content.parts},
    {'role':'user',
     'parts': function_responses}
]

# 質問応答
response = model.generate_content(messages)
print(response.text)

東京は20度、大阪は10度です。


In [104]:
# 会話履歴の作成
messages = [
    {'role':'user',
     'parts': [prompt]},
    {'role':'model',
     'parts': response.candidates[0].content.parts},
    {'role':'user',
     'parts': function_responses[::-1]}
]

# 質問応答
response = model.generate_content(messages)
print(response.text)

東京は10度、大阪は20度です。
